<a href="https://colab.research.google.com/github/raruggie/GRRIEn/blob/main/tutorials/detecting-changes-in-sentinel-1-imagery-pt-1/CEE609_data_download_preprocess.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@title Copyright 2020 The Earth Engine Community Authors { display-mode: "form" }
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

In [9]:
#attach to Google Drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [10]:
import ee
 
# Trigger the authentication flow.
ee.Authenticate()
 
# Initialize the library.
ee.Initialize()

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://code.earthengine.google.com/client-auth?scopes=https%3A//www.googleapis.com/auth/earthengine%20https%3A//www.googleapis.com/auth/devstorage.full_control&request_id=3u0K0TPtY7AmStdGT_5RKl4liZlggRbF-wpmDRq7mxA&tc=od3aSaZWfABRaAiX73nDEmo9fb5mEMRpZfvLS30HxaE&cc=qXuDlGA8NIot2qR5j2MTqINNdeVge9J6EZzRz-yHEng

The authorization workflow will generate a code, which you should paste in the box below.
Enter verification code: 4/1AWgavdeyEhDk3IL9Dc66JZ6-0Vx_TWEzoTb7gRqpWc6sezwJY4nxGsM1YK0

Successfully saved authorization token.


In [11]:
#@title Install packages for Google Colab

%%capture
!pip install rasterio
!pip install earthpy
import os
import itertools
import matplotlib.pyplot as plt
import rasterio as rio
from rasterio.plot import plotting_extent
import numpy as np
import earthpy as et
import earthpy.plot as ep
from scipy.stats import norm, gamma, f, chi2
import IPython.display as disp
%matplotlib inline

In [12]:
#@title Function to convert ee array to panda df
# need this function for below
import pandas as pd

def ee_array_to_df(arr, list_of_bands):
    """Transforms client-side ee.Image.getRegion array to pandas.DataFrame."""
    df = pd.DataFrame(arr)

    # Rearrange the header.
    headers = df.iloc[0]
    df = pd.DataFrame(df.values[1:], columns=headers)

    # Remove rows without data inside.
    df = df[['longitude', 'latitude', 'time', *list_of_bands]].dropna()

    # Convert the data to numeric values.
    for band in list_of_bands:
        df[band] = pd.to_numeric(df[band], errors='coerce')

    # Convert the time field into a datetime.
    df['datetime'] = pd.to_datetime(df['time'], unit='ms')

    # Keep the columns of interest.
    df = df[['time','datetime',  *list_of_bands]]

    return df

In [13]:
#@title Functions for Exogenous Organic Matter Indices (EOMI) band indices
def addEOMI1(image):
    EVI = image.expression('(B11-B8A)/(B11+B8A)', {
        'B11' : image.select('B11'),
        'B8A' : image.select('B8A')}).rename('EVI')
    return image.addBands(EVI)


def EOMI1(B11, B8A):
    """Calculated band index for EOMI"""
    return (B11-B8A)/(B11+B8A)

def EOMI2(B12, B4):
    """Calculated band index for EOM2"""
    x =  (B12-B4)/(B12+B4)
    return x

def EOMI3(B11, B8A, B12, B4):
    """Calculated band index for EOM3"""
    x =  ((B11-B8A)+(B12-B4))/((B11+B8A+B12+B4))
    return x

def EOMI4(B11, B4):
    """Calculated band index for EOM4"""
    x =  (B11-B4)/(B11+B4)
    return x

def NBR2(B11, B12):
    """Calculated band index for NBR2"""
    x =  (B11-B12)/(B11+B12)
    return x

In [14]:
#@title Functions for messing with s2cloudless proability image collection:

# intialize parameters used in functions
CLOUD_FILTER = 90
CLD_PRB_THRESH = 50
NIR_DRK_THRESH = 0.15
CLD_PRJ_DIST = 1
BUFFER = 50

# Define a function to filter the SR and s2cloudless collections according to area of interest and date parameters, 
# then join them on the system:index property. The result is a copy of the SR collection where each image has a new 
# 's2cloudless' property whose value is the corresponding s2cloudless image
def get_s2_sr_cld_col(aoi, start_date, end_date):
    # Import and filter S2 SR.
    s2_sr_col = (ee.ImageCollection('COPERNICUS/S2_SR')
        .filterBounds(aoi)
        .filterDate(start_date, end_date)
        .filter(ee.Filter.lte('CLOUDY_PIXEL_PERCENTAGE', CLOUD_FILTER)))
    # Import and filter s2cloudless.
    s2_cloudless_col = (ee.ImageCollection('COPERNICUS/S2_CLOUD_PROBABILITY')
        .filterBounds(aoi)
        .filterDate(start_date, end_date))
    # Join the filtered s2cloudless collection to the SR collection by the 'system:index' property.
    return ee.ImageCollection(ee.Join.saveFirst('s2cloudless').apply(**{
        'primary': s2_sr_col,
        'secondary': s2_cloudless_col,
        'condition': ee.Filter.equals(**{
            'leftField': 'system:index',
            'rightField': 'system:index'
        })
    }))

# Define a function to add the s2cloudless probability layer and derived cloud mask as bands to an S2 SR image input.
def add_cloud_bands(img):
    # Get s2cloudless image, subset the probability band.
    cld_prb = ee.Image(img.get('s2cloudless')).select('probability')
    # Condition s2cloudless by the probability threshold value.
    is_cloud = cld_prb.gt(CLD_PRB_THRESH).rename('clouds')
    # Add the cloud probability layer and cloud mask as image bands.
    return img.addBands(ee.Image([cld_prb, is_cloud]))
  
# Define a function to add dark pixels, cloud projection, and identified shadows as bands to an S2 SR image input. 
# Note that the image input needs to be the result of the above add_cloud_bands function because it relies on knowing which pixels are 
# considered cloudy ('clouds' band)
def add_shadow_bands(img):
    # Identify water pixels from the SCL band.
    not_water = img.select('SCL').neq(6)
    # Identify dark NIR pixels that are not water (potential cloud shadow pixels).
    SR_BAND_SCALE = 1e4
    dark_pixels = img.select('B8').lt(NIR_DRK_THRESH*SR_BAND_SCALE).multiply(not_water).rename('dark_pixels')
    # Determine the direction to project cloud shadow from clouds (assumes UTM projection).
    shadow_azimuth = ee.Number(90).subtract(ee.Number(img.get('MEAN_SOLAR_AZIMUTH_ANGLE')));
    # Project shadows from clouds for the distance specified by the CLD_PRJ_DIST input.
    cld_proj = (img.select('clouds').directionalDistanceTransform(shadow_azimuth, CLD_PRJ_DIST*10)
        .reproject(**{'crs': img.select(0).projection(), 'scale': 100})
        .select('distance')
        .mask()
        .rename('cloud_transform'))
    # Identify the intersection of dark pixels with cloud shadow projection.
    shadows = cld_proj.multiply(dark_pixels).rename('shadows')
    # Add dark pixels, cloud projection, and identified shadows as image bands.
    return img.addBands(ee.Image([dark_pixels, cld_proj, shadows]))

# Define a function to assemble all of the cloud and cloud shadow components and produce the final mask.
def add_cld_shdw_mask(img):
    # Add cloud component bands.
    img_cloud = add_cloud_bands(img)
    # Add cloud shadow component bands.
    img_cloud_shadow = add_shadow_bands(img_cloud)
    # Combine cloud and shadow mask, set cloud and shadow as value 1, else 0.
    is_cld_shdw = img_cloud_shadow.select('clouds').add(img_cloud_shadow.select('shadows')).gt(0)
    # Remove small cloud-shadow patches and dilate remaining pixels by BUFFER input.
    # 20 m scale is for speed, and assumes clouds don't require 10 m precision.
    is_cld_shdw = (is_cld_shdw.focalMin(2).focalMax(BUFFER*2/20)
        .reproject(**{'crs': img.select([0]).projection(), 'scale': 20})
        .rename('cloudmask'))
    # Add the final cloud-shadow mask to the image.
    return img_cloud_shadow.addBands(is_cld_shdw)

# Define a function to apply the cloud mask to each image in the collection.
def apply_cld_shdw_mask(img):
    # Subset the cloudmask band and invert it so clouds/shadow are 0, else 1.
    not_cld_shdw = img.select('cloudmask').Not()

    # Subset reflectance bands and update their masks, return the result.
    return img.select('B.*').updateMask(not_cld_shdw)

# Define functions to display image and mask component layers.
# Folium will be used to display map layers. Import folium and define a method to display Earth Engine image tiles.
# Import the folium library.
import folium

# Define a method for displaying Earth Engine image tiles to a folium map.
def add_ee_layer(self, ee_image_object, vis_params, name, show=True, opacity=1, min_zoom=0):
    map_id_dict = ee.Image(ee_image_object).getMapId(vis_params)
    folium.raster_layers.TileLayer(
        tiles=map_id_dict['tile_fetcher'].url_format,
        attr='Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
        name=name,
        show=show,
        opacity=opacity,
        min_zoom=min_zoom,
        overlay=True,
        control=True
        ).add_to(self)

# Add the Earth Engine layer method to folium.
folium.Map.add_ee_layer = add_ee_layer

# Define a function to display all of the cloud and cloud shadow components to an interactive Folium map. 
# The input is an image collection where each image is the result of the add_cld_shdw_mask function defined previously
def display_cloud_layers(col):
    # Mosaic the image collection.
    img = col.mosaic()
    # Subset layers and prepare them for display.
    clouds = img.select('clouds').selfMask()
    shadows = img.select('shadows').selfMask()
    dark_pixels = img.select('dark_pixels').selfMask()
    probability = img.select('probability')
    cloudmask = img.select('cloudmask').selfMask()
    cloud_transform = img.select('cloud_transform')
    # Create a folium map object.
    center = AOI.centroid(10).coordinates().reverse().getInfo()
    m = folium.Map(location=center, zoom_start=13)
    # Add layers to the folium map.
    m.add_ee_layer(img,{'bands': ['B4', 'B3', 'B2'], 'min': 0, 'max': 2500, 'gamma': 1.1},'S2 image', True, 1, 9)
    m.add_ee_layer(probability,{'min': 0, 'max': 100},'probability (cloud)', False, 1, 9)
    m.add_ee_layer(clouds, {'palette': 'e056fd'}, 'clouds', False, 1, 9)
    m.add_ee_layer(cloud_transform, {'min': 0, 'max': 1, 'palette': ['white', 'black']}, 'cloud_transform', False, 1, 9)
    m.add_ee_layer(dark_pixels, {'palette': 'orange'}, 'dark_pixels', False, 1, 9)
    m.add_ee_layer(shadows, {'palette': 'yellow'}, 'shadows', False, 1, 9)
    m.add_ee_layer(cloudmask, {'palette': 'orange'},'cloudmask', True, 0.5, 9)
    # Add a layer control panel to the map.
    m.add_child(folium.LayerControl())
    # Display the map.
    display(m)

In [15]:
#@title Geojson for field aois
# AHS and DC
geoJSON = {
  "type": "FeatureCollection",
  "features": [
    {
      "type": "Feature",
      "properties": {},
      "geometry": {
        "coordinates": [
          [
            [
              -73.31939702284171,
              44.15031909900094
            ],
            [
              -73.31499969430045,
              44.150539958983614
            ],
            [
              -73.3151316141566,
              44.15101322759219
            ],
            [
              -73.31442804159038,
              44.151107880858405
            ],
            [
              -73.3146479080171,
              44.15199130402354
            ],
            [
              -73.31526353401328,
              44.15240146028401
            ],
            [
              -73.31605505315063,
              44.15230680909241
            ],
            [
              -73.31631889286288,
              44.153474163180505
            ],
            [
              -73.31517558744216,
              44.153663461667435
            ],
            [
              -73.31561532029615,
              44.15543021826613
            ],
            [
              -73.31675862571684,
              44.155998093082275
            ],
            [
              -73.31715438528579,
              44.155998093082275
            ],
            [
              -73.31737425171306,
              44.15498853407544
            ],
            [
              -73.3179019311376,
              44.15473614162491
            ],
            [
              -73.31768206471087,
              44.153821209942976
            ],
            [
              -73.3179019311376,
              44.15334796385173
            ],
            [
              -73.31886934341718,
              44.15284316383796
            ],
            [
              -73.31979278241064,
              44.1520228546064
            ],
            [
              -73.31939702284171,
              44.15031909900094
            ]
          ]
        ],
        "type": "Polygon"
      }
    },
    {
      "type": "Feature",
      "properties": {},
      "geometry": {
        "coordinates": [
          [
            [
              -73.3582447745077,
              44.143810086322816
            ],
            [
              -73.35799464344306,
              44.14345635041016
            ],
            [
              -73.35772490610948,
              44.143241279968976
            ],
            [
              -73.35715546062814,
              44.143198265786594
            ],
            [
              -73.35711050440575,
              44.14287565842099
            ],
            [
              -73.35678082544304,
              44.14277887586772
            ],
            [
              -73.356900708702,
              44.142402497762504
            ],
            [
              -73.35691569410945,
              44.14207988604832
            ],
            [
              -73.35684076707214,
              44.14184330300441
            ],
            [
              -73.35685575247962,
              44.14162822668689
            ],
            [
              -73.35720041685052,
              44.1414991805205
            ],
            [
              -73.35751511040576,
              44.14128410294927
            ],
            [
              -73.35773989151696,
              44.140595849457185
            ],
            [
              -73.35785977477666,
              44.14006889995392
            ],
            [
              -73.35717044603558,
              44.139746275487965
            ],
            [
              -73.35066677921822,
              44.1403700145315
            ],
            [
              -73.35101062208136,
              44.142072309106425
            ],
            [
              -73.3512204177851,
              44.143212196086125
            ],
            [
              -73.35270397311872,
              44.14305089266176
            ],
            [
              -73.35306684447445,
              44.145149409871834
            ],
            [
              -73.35327662997008,
              44.14668946420335
            ],
            [
              -73.3535313818962,
              44.1480658210873
            ],
            [
              -73.35230257848795,
              44.14815184232668
            ],
            [
              -73.3525123741917,
              44.14942064104943
            ],
            [
              -73.3560964692142,
              44.14912893206903
            ],
            [
              -73.35819442625233,
              44.148935387307944
            ],
            [
              -73.35862900306724,
              44.14884936720992
            ],
            [
              -73.35870393010455,
              44.14861281129507
            ],
            [
              -73.35876387173367,
              44.14820421248106
            ],
            [
              -73.35892871121575,
              44.148021417359104
            ],
            [
              -73.359243404771,
              44.147935395929636
            ],
            [
              -73.35961803995684,
              44.14784937437426
            ],
            [
              -73.35984282106803,
              44.14772034180669
            ],
            [
              -73.35987818391523,
              44.14697151244681
            ],
            [
              -73.35987818391523,
              44.14645537280754
            ],
            [
              -73.35996809635998,
              44.145724167260084
            ],
            [
              -73.3600879796197,
              44.14497144620606
            ],
            [
              -73.36011795043387,
              44.14428323569982
            ],
            [
              -73.359248796804,
              44.143831593195046
            ],
            [
              -73.3582447745077,
              44.143810086322816
            ]
          ]
        ],
        "type": "Polygon"
      }
    }
  ]
}
# general Panton area
geoJSON_gen = {
  "type": "FeatureCollection",
  "features": [
    {
      "type": "Feature",
      "properties": {},
      "geometry": {
        "coordinates": [
          [
            [
              -73.39192033862791,
              44.17172464979126
            ],
            [
              -73.39192033862791,
              44.11449266900607
            ],
            [
              -73.28488148069843,
              44.11449266900607
            ],
            [
              -73.28488148069843,
              44.17172464979126
            ],
            [
              -73.39192033862791,
              44.17172464979126
            ]
          ]
        ],
        "type": "Polygon"
      }
    }
  ]
}
# save geojson as ee geometery class

# general panton area
coords_gen = geoJSON_gen['features'][0]['geometry']['coordinates']
aoi_gen = ee.Geometry.Polygon(coords_gen)

# AHS and DC fields
coords_DC = geoJSON['features'][0]['geometry']['coordinates']
coords_AHS = geoJSON['features'][1]['geometry']['coordinates']
aoi_DC = ee.Geometry.Polygon(coords_DC)
aoi_AHS = ee.Geometry.Polygon(coords_AHS)

Sentinel-2 Cloud Masking with s2cloudless
https://developers.google.com/earth-engine/tutorials/community/sentinel-2-s2cloudless

In [45]:
#@title DC 2019 pre

# Define collection filter and cloud mask parameters
AOI = aoi_DC
START_DATE = '2019-09-29'
END_DATE = '2019-10-11'

# get pre collection

c1 = get_s2_sr_cld_col(AOI, START_DATE, END_DATE)

# Add cloud and cloud shadow component bands to each image and then apply the mask to each image. 
# Reduce the collection by median (in your application, you might consider using medoid reduction to build
# a composite from actual data values, instead of per-band statistics).
# Actually ended up using getRegion because the image resulting from .median.clip was not working

img = c1.map(add_cld_shdw_mask).map(apply_cld_shdw_mask).median().clip(AOI) # dont use this one for now
img_for_band_names = c1.map(add_cld_shdw_mask).map(apply_cld_shdw_mask).getRegion(aoi_AHS, 20).getInfo() # note I have no idea how t control which image I get. 

# export image to pandas dataframe

band_names = list(np.concatenate([item[-12:] for item in img_for_band_names[:1]])) # get bandnames of all bands to use in function below
band_names
# img_pdf = ee_array_to_df(img,band_names) # note this function only works for an image list that was created using getRegion.getinfo. THIS IS THE ONLY WAY I COULD FIGURE OUT HOW TO DO THIS


# i still want just one image, not the median. My idea is to sum up all the pixels in the mask band for each image
# in the collection, then sort by ascending on the sums, so at the top of the list is the image with the 
# least number of masked pixels. Having one image lets me determine prior rainfall to the date of image aquisition to 
# better replicate the results of Dodin et al. 2021

# Compute and add multiple bands, 1,2,4,5, note band 3 needs a different method since it is not just a simple normalized difference between two bands.
# idx1 = img.normalizedDifference(['B11', 'B8A']).rename('EOMI_1')
# idx2 = img.normalizedDifference(['B12', 'B4']).rename('EOMI_2')
# bandNameExp = '((b("B11") - b("B8A")) + (b("B12") - b("B4"))) / (b("B11") + b("B8A") + b("B12") + b("B4"))'
# idx3 = img.expression(bandNameExp).rename('EOMI_3')
# idx4 = img.normalizedDifference(['B11', 'B4']).rename('EOMI_4')
# idx5 = img.normalizedDifference(['B11', 'B12']).rename('EOMI_5')
# newBands = ee.Image([idx1, idx2, idx3, idx4, idx5])

# # add bands to image
# img_EOMI = img.addBands(newBands).getInfo()



# ee_array_to_df(a19_pre_list,['B12', 'B11', 'B8A', 'B4'])
# export image as csv to drive
# ee.batch.Export.image.toDrive(img_EOMI).start()


['B1', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B8', 'B8A', 'B9', 'B11', 'B12']

In [ ]:
!pwd

/content


In [ ]:
#@title Map the image from above code cell using Folium
# Create a folium map object.
center = AOI.centroid(10).coordinates().reverse().getInfo()
m = folium.Map(location=center, zoom_start=14)

# Add layers to the folium map.
m.add_ee_layer(s2_sr_median,
                {'bands': ['B4', 'B3', 'B2'], 'min': 0, 'max': 2500, 'gamma': 1.1},
                'S2 cloud-free mosaic', True, 1, 9)

# Add a layer control panel to the map.
m.add_child(folium.LayerControl())

# Display the map.
display(m)

Clearly the cloudy cover percentage property doesn't apply to the aoi because when  Iset ascending to false in the sort function the image has less clouds of AHS. 

Moving along, will come back to cloudy pixel issue later. So assume you have the least cloudy images available to you...

In [ ]:
#@title Find out dates of best images using geemap
# # find out dates of best images
# print(geemap.image_props(s2_col_pre).get('IMAGE_DATE').getInfo())
# print(geemap.image_props(s2_col_post).get('IMAGE_DATE').getInfo())
# # print(geemap.image_props(s2_col_pre).getInfo())

In [ ]:
#@title Left over code from S2 cloud proability workflow

# # apply functionv to merge S2 SR and S2 cloud probability image collections
# # this adds a 's2cloudless' property whose value is the corresponding s2cloudless image to the images in the S2 SR image collection
# pre = get_s2_sr_cld_col(AOI, START_DATE, END_DATE).first()
# landsat_props = geemap.image_props(pre)
# print(landsat_props.getInfo())

# # # Map the add_cld_shdw_mask function over the collection to add mask component bands to each image.
# # s2_sr_cld_col_eval_disp = pre.map(add_cld_shdw_mask)
# s2_sr_cld_col_eval_disp = add_cld_shdw_mask(pre)
# landsat_props = geemap.image_props(s2_sr_cld_col_eval_disp)
# print(landsat_props.getInfo())

# # sort by 


# 
# print('Collection without properties', s2_sr_cld_col_eval_disp)
# lst = pre.getRegion(aoi_AHS, 20).getInfo()
# lst[:1]
# names = [item[0] for item in lst]
# set(names)



# # Display mask component layers
# # Map the add_cld_shdw_mask function over the collection to add mask component bands to each image, then display the results.
# # Give the system some time to render everything, it should take less than a minute.
# s2_sr_cld_col_eval_disp = pre.map(add_cld_shdw_mask)
# # display_cloud_layers(s2_sr_cld_col_eval_disp)

# # filter images in collections for the ones with the lease clouds
# # to do this I will look at the sum of the three new bands for each image, clouds, dark_pixels, and shadows
# # the images with the lowest sums will have the most data over the fields, and will be the best images to use
# lst = s2_sr_cld_col_eval_disp.getRegion(aoi_AHS, 20).getInfo()
# # print(lst[:5])
# # names = [item[0] for item in lst]
# # set(names)

# # split up list of list by unqiue values in ID column
# # d = [list(x) for _,x in itertools.groupby(lst,lambda x:x[0])]
# # d[:5]

# # convert to pandas datafame using function from above
# pdf = ee_array_to_df(lst,['clouds', 'dark_pixels', 'shadows'])

# # calcualte the number of no-good pixels in each image
# pdf['pixels'] = 1 # create cell to check if all dates have the same number of pixels
# pdf['sum'] = pdf.iloc[:, 2:5].sum(axis=1) # remeber panads last index in not inclusive
# pdf = pdf.rename(columns={'datetime': 'date'})
# # pdf.dtypes # check column classes
# pdf['date'] = pd.to_datetime(pdf['date']).dt.date
# pdf = pdf.drop('time', axis=1)
# pdf = pdf.groupby('date')['clouds', 'dark_pixels', 'shadows', 'sum', 'pixels'].sum()
# print(pdf.head(20))

# # check that pixel count equal field size - AHS is 14.16 ha
# 1818*(20)*0.0001 # number if pixels * spatial resolution in m2 * conversion from m2 to ha

In [26]:
# image collection, filter over AHS aoi
col = ee.ImageCollection('COPERNICUS/S2_SR').filterBounds(aoi_gen)
# select bands
col = col.select(['B12', 'B11', 'B8A', 'B4'])
# filter date - start with general date to narrow collection
col = col.filterDate('2018-10-01','2020-11-01')
a19_pre_list
# get the data for the pixels over AHS field
# col_list = col.getRegion(aoi_AHS, 20).getInfo() # not sure what scale to use here, assume it is of largest resolution of the listed bands
# also, need to filter farther first as this is too big for nlist object in py...
# a18_pre_list=col.filterDate('2018-10-01','2018-10-15').getRegion(aoi_AHS, 20).getInfo()
# a18_post_list=col.filterDate('2018-10-17', '2018-10-30').getRegion(aoi_AHS, 20).getInfo()
# this lists no bands in collection, proably no images in 2018
# trying 2019
# a19_pre_list=col.filterDate('2019-10-01','2019-10-15').getRegion(aoi_AHS, 20).getInfo()
# a19_post_list=col.filterDate('2019-10-17', '2019-10-30').getRegion(aoi_AHS, 20).getInfo()
# # preview resulting list
# len(a19_pre_list)
# a19_pre_list[:5]
# # convert to pandas datafame using function from above
# a19_pre_df = ee_array_to_df(a19_pre_list,['B12', 'B11', 'B8A', 'B4'])
# a19_post_df = ee_array_to_df(a19_post_list,['B12', 'B11', 'B8A', 'B4'])
# # # look at df
# print(a19_pre_df.head(20))
# print(a19_post_df.head(20))
# there are 5 images for pre and 5 for post
# need to do some sort of cloud analysis
# S2A product has a 60meter colud mask band, but the bands I selected above are 10 and 20m. I think 60 


# # calculate EOI indexs from Dodin et al. 2021
# x=a19_pre_df
# y=a19_post_df
# x['EOMI1']=(x['B11']-x['B8A'])/(x['B11']+x['B8A']) # why doesn't this work: a19_pre_df.apply(lambda x: EOMI1(a19_pre_df['B11'], a19_pre_df['B8A']), axis=1) 
# x['EOMI2']=(x['B12']-x['B4'])/(x['B12']+x['B4'])
# x['EOMI3']=((x['B11']-x['B8A'])+(x['B12']-x['B4']))/(x['B11']+x['B8A']+x['B12']+x['B4'])
# x['EOMI4']=(x['B11']-x['B4'])/(x['B11']+x['B4'])
# x['NBR2']=(x['B11']-x['B12'])/(x['B11']+x['B12'])
# y['EOMI1']=(y['B11']-y['B8A'])/(y['B11']+y['B8A']) 
# y['EOMI2']=(y['B12']-y['B4'])/(y['B12']+y['B4'])
# y['EOMI3']=((y['B11']-y['B8A'])+(y['B12']-y['B4']))/(y['B11']+y['B8A']+y['B12']+y['B4'])
# y['EOMI4']=(y['B11']-y['B4'])/(y['B11']+y['B4'])
# y['NBR2']=(y['B11']-y['B12'])/(y['B11']+y['B12'])
# plot histograms
# x.iloc[:,-5:].hist()
# y.iloc[:,-5:].hist()
# will filter the stack into multiple variables for each year of manure application
# a18_pre=col.filterDate('2018-10-01','2018-10-15') 
# a18_post=col.filterDate('2018-10-17', '2018-10-30')

[['id', 'longitude', 'latitude', 'time', 'B12', 'B11', 'B8A', 'B4'],
 ['20191003T155101_20191003T155823_T18TXP',
  -73.35741424801266,
  44.13988929508204,
  1570118462144,
  2706,
  3059,
  6271,
  6180],
 ['20191005T154109_20191005T154756_T18TXP',
  -73.35741424801266,
  44.13988929508204,
  1570290664841,
  1823,
  2616,
  2966,
  1554],
 ['20191008T155139_20191008T155137_T18TXP',
  -73.35741424801266,
  44.13988929508204,
  1570550461130,
  1413,
  1866,
  1963,
  1188],
 ['20191010T154151_20191010T154457_T18TXP',
  -73.35741424801266,
  44.13988929508204,
  1570722666726,
  2055,
  2610,
  2412,
  1560],
 ['20191013T155211_20191013T155920_T18TXP',
  -73.35741424801266,
  44.13988929508204,
  1570982463067,
  2036,
  2517,
  2215,
  1520],
 ['20191003T155101_20191003T155823_T18TXP',
  -73.35723458495583,
  44.13988929508204,
  1570118462144,
  2706,
  3059,
  6271,
  6196],
 ['20191005T154109_20191005T154756_T18TXP',
  -73.35723458495583,
  44.13988929508204,
  1570290664841,
  182

In [ ]:
# # Copied from GEE website for testing

# # Import the MODIS land cover collection.
# lc = ee.ImageCollection('MODIS/006/MCD12Q1')

# # Import the MODIS land surface temperature collection.
# lst = ee.ImageCollection('MODIS/006/MOD11A1')

# # Import the USGS ground elevation image.
# elv = ee.Image('USGS/SRTMGL1_003')

# # Initial date of interest (inclusive).
# i_date = '2017-01-01'

# # Final date of interest (exclusive).
# f_date = '2020-01-01'

# # Selection of appropriate bands and dates for LST.
# lst = lst.select('LST_Day_1km', 'QC_Day').filterDate(i_date, f_date)

# # Define the urban location of interest as a point near Lyon, France.
# u_lon = 4.8148
# u_lat = 45.7758
# u_poi = ee.Geometry.Point(u_lon, u_lat)

# # Define the rural location of interest as a point away from the city.
# r_lon = 5.175964
# r_lat = 45.574064
# r_poi = ee.Geometry.Point(r_lon, r_lat)

# scale = 1000  # scale in meters

# # Print the elevation near Lyon, France.
# elv_urban_point = elv.sample(u_poi, scale).first().get('elevation').getInfo()
# print('Ground elevation at urban point:', elv_urban_point, 'm')

# # Calculate and print the mean value of the LST collection at the point.
# lst_urban_point = lst.mean().sample(u_poi, scale).first().get('LST_Day_1km').getInfo()
# print('Average daytime LST at urban point:', round(lst_urban_point*0.02 -273.15, 2), '°C')

# # Print the land cover type at the point.
# lc_urban_point = lc.first().sample(u_poi, scale).first().get('LC_Type1').getInfo()
# print('Land cover value at urban point is:', lc_urban_point)

# # Get the data for the pixel intersecting the point in urban area.
# lst_u_poi = lst.getRegion(u_poi, scale).getInfo()

# # Get the data for the pixel intersecting the point in rural area.
# lst_r_poi = lst.getRegion(r_poi, scale).getInfo()

# Preview the result.
print(type(lst_u_poi))
# lst_u_poi[:5]

<class 'list'>


In [ ]:
# Import the Folium library.
import folium

# Define a method for displaying Earth Engine image tiles to folium map.
def add_ee_layer(self, ee_image_object, vis_params, name):
  map_id_dict = ee.Image(ee_image_object).getMapId(vis_params)
  folium.raster_layers.TileLayer(
    tiles = map_id_dict['tile_fetcher'].url_format,
    attr = 'Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
    name = name,
    overlay = True,
    control = True
  ).add_to(self)

# Add EE drawing method to folium.
folium.Map.add_ee_layer = add_ee_layer

In [ ]:
# print image

# select bands
img=S2_AHS_2018_stack.first().select(['B4', 'B3', 'B2'])

# Get 2-d pixel array for AOI - returns feature with 2-D pixel array as property per band.
band_arrs = img.sampleRectangle(region=aoi_AHS)

# Get individual band arrays.
band_arr_b4 = band_arrs.get('B4')
band_arr_b5 = band_arrs.get('B3')
band_arr_b6 = band_arrs.get('B2')

# Transfer the arrays from server to client and cast as np array.
np_arr_b4 = np.array(band_arr_b4.getInfo())
np_arr_b5 = np.array(band_arr_b5.getInfo())
np_arr_b6 = np.array(band_arr_b6.getInfo())
print(np_arr_b4.shape)
print(np_arr_b5.shape)
print(np_arr_b6.shape)

# Expand the dimensions of the images so they can be concatenated into 3-D.
np_arr_b4 = np.expand_dims(np_arr_b4, 2)
np_arr_b5 = np.expand_dims(np_arr_b5, 2)
np_arr_b6 = np.expand_dims(np_arr_b6, 2)
print(np_arr_b4.min())
print(np_arr_b4.max())
print(np_arr_b4.shape)
print(np_arr_b5.shape)
print(np_arr_b6.shape)

# Stack the individual bands to make a 3-D array.
rgb_img = np.concatenate((np_arr_b6, np_arr_b5, np_arr_b4), 2)
print(rgb_img.shape)

# Scale the data to [0, 255] to show as an RGB image.
rgb_img_test = (255*((rgb_img-rgb_img.min())/(rgb_img.max()-rgb_img.min()))).astype('uint8')
plt.imshow(rgb_img_test)
plt.show()


EEException: ignored

In [ ]:
S1_DC_2019_stack = ee.ImageCollection('COPERNICUS/S1_GRD').filterBounds(aoi_DC).filterDate(ee.Date('2019-10-01'), ee.Date('2019-10-31')) 

Notice that we have clipped the images to our _aoi_ so as not to work with the entire swath. To confirm that we have an image, we list its band names, fetching the result from the GEE servers with the _getInfo()_ class method:

In [ ]:
ee.Image(S1_DC_2019_stack.first()).bandNames().getInfo()

# ['VV', 'VH', 'angle']
# I'm not sure why HH and HV are not here

['VV', 'VH', 'angle']

and display the VV band of the decibel version using the _getThumbURL()_ method and IPython's _display_ module. The float intensities $I$ are generally between 0 and 1, so we stretch the decibel image $10\log_{10}(I)$ from $-20$ to $0$:

In [ ]:
url = S1_DC_2019_stack.first().clip(aoi_DC).select('VV').getThumbURL({'min': -20, 'max': 0})
disp.Image(url=url, width=200)

This is fine, but a little boring. We can use _folium_ to project onto a map for geographical context. The _folium_ _Map()_ constructor wants its _location_ keyword in long-lat rather than lat-long, so we do a list reverse in the first line:

In [ ]:
location = aoi.centroid().coordinates().getInfo()[::-1]

# Make an RGB color composite image (VV,VH,VV/VH).
rgb = ee.Image.rgb(ffa_db.select('VV'),
                   ffa_db.select('VH'),
                   ffa_db.select('VV').divide(ffa_db.select('VH')))

# Create the map object.
m = folium.Map(location=location, zoom_start=12)

# Add the S1 rgb composite to the map object.
m.add_ee_layer(rgb, {'min': [-20, -20, 0], 'max': [0, 0, 2]}, 'FFA')

# Add a layer control panel to the map.
m.add_child(folium.LayerControl())

# Display the map.
display(m)

### Pixel distributions

In order to examine the statistics of the pixels in this image empirically, we'll need samples from a featureless (textureless) spatial subset. Here is a polygon covering the triangular wooded area just east of the north-south runway:

In [ ]:
geoJSON = {
  "type": "FeatureCollection",
  "features": [
    {
      "type": "Feature",
      "properties": {},
      "geometry": {
        "type": "Polygon",
        "coordinates": [
          [
            [
              8.534317016601562,
              50.021637833966786
            ],
            [
              8.530540466308594,
              49.99780882512238
            ],
            [
              8.564186096191406,
              50.00663576154257
            ],
            [
              8.578605651855469,
              50.019431940583104
            ],
            [
              8.534317016601562,
              50.021637833966786
            ]
          ]
        ]
      }
    }
  ]
}
coords = geoJSON['features'][0]['geometry']['coordinates']
aoi_sub = ee.Geometry.Polygon(coords)

Using standard reducers from the GEE library we can easily calculate a histogram and estimate the first two moments (mean and variance) of the pixels in the polygon _aoi\_sub_ , again retrieving the results from the servers with _getInfo()_ .

In [ ]:
hist = ffa_fl.select('VV').reduceRegion(
    ee.Reducer.fixedHistogram(0, 0.5, 500),aoi_sub).get('VV').getInfo()
mean = ffa_fl.select('VV').reduceRegion(
    ee.Reducer.mean(), aoi_sub).get('VV').getInfo()
variance = ffa_fl.select('VV').reduceRegion(
    ee.Reducer.variance(), aoi_sub).get('VV').getInfo()

Here is a plot of the (normalized) histogram using _numpy_ and _matplotlib_ :

In [ ]:
a = np.array(hist)
x = a[:, 0]                 # array of bucket edge positions
y = a[:, 1]/np.sum(a[:, 1]) # normalized array of bucket contents
plt.grid()
plt.plot(x, y, '.')
plt.show()

The above histogram is in fact a _gamma probability density distribution_ 

$$
p_{\gamma;\alpha,\beta}(x) = {1\over \beta^\alpha\Gamma(\alpha)}x^{\alpha-1}e^{-x/\beta},\quad {\rm mean}(x) = \alpha\beta,\quad {\rm var}(x) = \alpha\beta^2 \tag{1.1}
$$
where
$$
\Gamma(\alpha) = \int_0^\infty z^{\alpha-1}e^{-z} dz.
$$

The parameters are in this case $\alpha = 5$ and $\beta = {\mu}/\alpha$, where $\mu$ is the estimated mean value we just determined with _ee.Reducer.mean()_. 
This can easily be verified by plotting the gamma distribution _gamma.pdf()_ and overlaying it onto the histogram. Since the bucket widths are 0.001, we have to divide the plot by 1000.

In [ ]:
alpha = 5
beta = mean/alpha
plt.grid()
plt.plot(x, y, '.', label='data')
plt.plot(x, gamma.pdf(x, alpha, 0, beta)/1000, '-r', label='gamma')
plt.legend()
plt.show()

In order to understand just why this is the case, let's take a step back and consider how the pixels were generated.


### Single look complex (SLC) SAR measurements
The Sentinel-1 platform is a dual polarimetric synthetic aperture radar system, emitting radar microwaves in the C-band with one polarization (vertical in most cases) and recording both vertical and horizontal reflected polarizations. This is represented mathematically as

$$
\pmatrix{E_v^b\cr E_h^b} = {e^{-{\bf i}rk}\over r}\pmatrix{S_{vv} & S_{vh}\cr S_{hv} & S_{hh}}\pmatrix{E_v^i\cr 0}. \tag{1.2}
$$

The incident, vertically polarized radar signal $\pmatrix{E_v^i\cr 0}$ is transformed by a complex _scattering matrix_ $\pmatrix{S_{vv} & S_{vh}\cr S_{hv} & S_{hh}}$ into the backscattered signal $\pmatrix{E_v^b\cr E_h^b}$ having both vertical and horizontal polarization components. The exponent term accounts for the phase shift due to the return distance $r$ from target to sensor, where $k$ is the wave number, $k=2\pi/\lambda$. From measurement of the backscattered radiation at the sensor, two of the four complex scattering matrix elements can be derived and processed into two-dimensional (slant range $\times$ azimuth) arrays, comprising the so-called _single look complex_ image. Written as a complex vector, the two derived elements are
 
$$
S = \pmatrix{S_{vv}\cr S_{vh}}.          \tag{1.3}
$$

We write the complex transpose of the vector $S$ as $S^\dagger = (S_{vv}^*\ S_{vh}^*)$, where the $^*$ denotes complex conjugation. The inner product of $S$ with itself is the total power (also referred to as the _span_ image)

$$
P = S^\dagger S = (S_{vv}^*\ S_{vh}^*)\pmatrix{S_{vv}\cr S_{vh}} = |S_{vv}|^2 + |S_{vh}|^2 \tag{1.4}
$$

and the outer product is the (dual pol) _covariance matrix image_

$$
C2 = SS^\dagger = \pmatrix{S_{vv}\cr S_{vh}}(S_{vv}^*\ S_{vh}^*) = \pmatrix{|S_{vv}|^2 & S_{vv}^*S_{vh} \cr S_{vh}^*S_{vv} & |S_{vh}|^2}. \tag{1.5}
$$

The diagonal elements are real numbers, the off-diagonal elements are complex conjugates of each other and contain the relative phases of the $S_{vv}$ and $S_{vh}$ components. The off-diagonal elements are not available for S1 archived imagery in GEE, so that if we nevertheless choose to represent the data in covariance matrix form, the matrix is diagonal: 

$$
C2 = \pmatrix{|S_{vv}|^2 & 0 \cr 0 & |S_{vh}|^2}, \tag{1.6a}
$$

In terms of radar scattering cross sections (sigma nought),

$$
C2 = {1\over 4\pi}\pmatrix{\sigma^o_{vv}  & 0 \cr 0 & \sigma^o_{vh}}. \tag{1.6b}
$$


### Speckle

The most striking characteristic of SAR images, when compared to their visual/infrared
counterparts, is the disconcerting _speckle_ effect which makes visual interpretation very
difficult. Speckle gives the appearance of random noise, but
it is actually a deterministic consequence of the coherent nature of the radar signal.

For single polarization transmission and reception, e.g., vertical-vertical ($vv$), the received SLC signal can be modelled in the form

$$
S_{vv} = {|S^a_{vv}|\over\sqrt{n}}\sum_{k=1}^n e^{{\bf i}\phi_k}, \tag{1.7}
$$

where $|S^a_{vv}|$ is the overall amplitude characterizing the signal scattered from the  area covered by a single pixel, e.g., $10\times 10\ m^2$ for our S1 data,  with the phase set equal to zero for convenience. The effects of randomly distributed scatterers within the irradiated area, with dimensions  of the order of the incident wavelength  5.6 cm (for Sentinel-1), add coherently and introduce a change in phase of the  received signal. This is indicated by the sum term in the above equation. The effect varies from pixel to pixel and gives rise to speckle. 

If we expand Eq. (1.7) into its real and imaginary parts, we can understand it better:

$$
S_{vv} = {|S^a_{vv}|\over\sqrt{n}}\sum_{k=1}^n e^{{\bf i}\phi_k} = {|S^a_{vv}|\over\sqrt{n}}\left(\sum_k\cos\phi_k + {\bf i}\sum_k\sin\phi_k\right) =  {|S^a_{vv}|\over\sqrt{n}}(x + {\bf i}y) \tag{1.8}
$$

where

$$
x = \sum_k\cos\phi_k, \quad y = \sum_k\sin\phi_k.
$$

Because the phase shifts $\phi_k$ are randomly and uniformly distributed, the variables $x$ and $y$ are sums of identically distributed cosine and sine terms respectively. The __Central Limit Theorem__ of statistics then says that $x$ and $y$ will have a normal distribution with zero mean and variance $\sigma^2 =n/2$ in the limit of large number $n$ of scatterers. We can verify this with a simple piece of code in which we set $n=10000$:

In [ ]:
def X(n):
    return np.sum(np.cos(4*np.pi*(np.random.rand(n)-0.5)))/np.sqrt(n/2)

n= 10000
Xs = [X(n) for i in range(10000)]
y, x = np.histogram(Xs, 100, range=[-5,5])
plt.plot(x[:-1], y/1000, 'b.', label='simulated data')
plt.plot(x, norm.pdf(x), '-r', label='normal distribution')
plt.grid()
plt.legend()
plt.show()

Furthermore, $x$ and $y$ are uncorrelated since, in the expression for covariance of $x$ and $y$, the sums of products of cosine and sine terms cancel to zero. This means that  $x + {\bf i}y$, and hence the observed single look complex signal $S_{vv}$ (see Eq. (1.8)), has a _complex normal distribution_ .

Now what about the pixels values in the Sentinel-1 VV intensity images? They are given by the square of the amplitude of $S_{vv}$,

$$
|S_{vv}|^2 = S_{vv}S^*_{vv} = {|S^a_{vv}|^2\over n}(x^2+y^2). \tag{1.9}
$$

(Actually averages of the above, as we'll see later.) We can write this in the form

$$
|S_{vv}|^2 =  {|S^a_{vv}|^2\over n}{n\over 2}\left({x^2\over n/2}+{y^2\over n/2}\right) = |S^a_{vv}|^2{u\over 2}, \tag{1.10}
$$

where 

$$
u = \left({x^2\over n/2}+{y^2\over n/2}\right) \tag{1.11}
$$

is the sum of the squares of two variables with independent standard normal distributions. Applying the  

   - __Theorem:__ If the measurements $x_i,\ i=1\dots m$, are independent and standard normally distributed (i.e., with mean $0$ and variance $1$), then the variable $x=\sum_{i=1}^m x_i^2$ is  chi-square distributed with $m$ degrees of freedom, given by

$$
p_{\chi^2;m}(x)={1 \over 2^{m/2}\Gamma(m/2)} z^{(m-2)/2} e^{-x/2},\quad {\rm mean}(x)=m,\quad {\rm var}(x) = 2m. \tag{1.12}
$$

we see that $u$ is chi-square distributed with degrees of freedom $m=2$,

$$
p_u(u) = {1\over 2}e^{-u/2} \tag{1.13}
$$

since $\Gamma(1)=1$.

To simplify the notation, let $s=|S_{vv}|^2 $ and  $a=|S^a_{vv}|^2$. Then from (1.10)

$$
s = a{u\over 2} \tag{1.14}
$$

To get the distribution $p_s(s)$ of the observed signal from the distribution of $u$, we apply the standard transformation formula

$$
p_s(s) = p_u(u)\left|{du\over ds}\right| = {1\over 2}e^{-u/2}{2\over a} = {1\over a} e^{-s/a}. \tag{1.15}
$$

Compare this with the definition of the _exponential probability distribution_

$$
p_{e;\beta}(x) = {1\over\beta}e^{-x/\beta},\quad {\rm mean}(x) = \beta,\quad {\rm var}(x) = \beta. \tag{1.16}
$$

We conclude that the measured intensity signal $s=|S_{vv}|^2$ has an exponential distribution with mean and variance equal to the underlying signal strength $a=|S^a_{vv}|^2$.

So far so good, however we still haven't quite characterized the statistics of the pixels in the intensity bands of the Sentinel-1 images. 


### Multi-look SAR statistics

Multi-look processing essentially corresponds to the averaging of neighborhood pixels with the objective
of reducing speckle and compressing the data. In practice, the averaging is often not performed in the
spatial domain, but rather in the frequency domain during range/azimuth compression of the received signal. 

Look averaging takes place at the cost of spatial resolution. The spatial resolution attainable with SAR satellite platforms  involves, among many other considerations, a compromise between azimuthal resolution and swath width, see  [Moreira et al. (2013)](https://elib.dlr.de/82313/) for a good discussion. In the Sentinel-1 _Interferometric Wide Swath_ acquisition mode, the pixels are about 20m $\times$ 4m (azimuth $\times$ range) in extent and the swath widths are about 250km. For the multi-looking procedure, five cells are incoherently averaged in the range direction to achieve approx. $20m \times 20m$ resolution. The pixels are then resampled to $10\times 10\ m^2$. (Note that spatial resolution is a measure of the system's ability to distinguish between adjacent targets while pixel spacing is the distance between adjacent pixels in an image, measured in metres.) The look averaging process, which we can symbolize using angular brackets as $\langle |S_{vv}|^2 \rangle$ or $\langle |S_{vh}|^2 \rangle$, has the desirable effect of reducing speckle (at the cost of resolution) in the intensity images. We can see this as follows, first quoting another well-known Theorem in statistics:

   - __Theorem:__ If the quantities $s_i,\ i=1\dots m,$ are independent and each have exponential distributions given by Eq. (1.16), then $x = \sum_{i=1}^m s_i$ has the gamma distribution Eq. (1.1) with $\alpha=m,\ \beta=a$. Its mean is $\alpha\beta =ma$ and its variance is $\alpha\beta^2 = ma^2.$

Again with the notation $s=|S_{vv}|^2 $ and  $a=|S^a_{vv}|^2$, if intensity measurements $s$ are summed over $m$ looks to give $\sum_{i=1}^m s_i$, then according to this Theorem  the sum (not the average!) will be gamma distributed with $\alpha= m$ and $\beta=a$, provided the $s_i$ are independent. The look-averaged image is

$$
\langle s\rangle = {1\over m}\sum_{i=1}^m s_i \tag{1.17}
$$

and its mean value is

$$
{\rm mean}(\langle s\rangle) = {1\over m}\sum_{i=1}^m {\rm mean}(s_i) = {1\over m}\sum_{i=1}^m a = a. \tag{1.18}
$$

Now we see that the histogram of the Sentinel-1 multi-look image $\langle s\rangle =\langle |S_{vv}|^2 \rangle$ follows a gamma distribution with the parameters 

$$
\alpha=m,\quad \beta' = {a\over m} = {{\rm mean}(\langle s\rangle)\over m}, \tag{1.19}
$$

as we demonstrated earlier with the measured histogram.

The covariance representation of the dual pol multilook images is 

$$
C2 = \pmatrix{\langle|S_{vv}|^2\rangle & 0 \cr 0 & \langle|S_{vh}|^2\rangle}. \tag{1.20}
$$


### Equivalent number of looks

The variance of $\langle s\rangle$ is given by

$$
{\rm var}(\langle s\rangle) = {1\over m^2}{\rm var}(\sum_{i=1}^m s_i)= {1\over m^2}ma^2 = {a^2\over m}, \tag{1.21}
$$

where we have used the fact that the variance of the gamma distribution is $\alpha\beta'^2=ma^2$. Thus the variance of the look-averaged image, the speckle effect, decreases inversely with the number of looks.


In practice, the neighborhood pixel intensities contributing to the look average will not be completely independent, but correlated to some extent. This is accounted for by defining an _equivalent number of looks_
(ENL) whose definition is motivated by Eq. (1.21), that is,

$$
{\rm ENL} = {a^2\over {\rm var}(\langle s\rangle)} = {{\rm mean}(\langle s\rangle)^2\over {\rm var}(\langle s\rangle)}.\tag{1.22}
$$

In general it will be smaller than $m$. Let's see what we get for our subset of the airport image:

In [ ]:
mean ** 2 / variance

The value given by the provider (ESA) for the IW mode imagery in the GEE archive is ENL = 4.4, an average over all swaths, so our spatial subset seems to be fairly representative.

### Outlook

Now we have a good idea of the statistics of the Sentinel-1 images on the GEE. In [Part 2](https://developers.google.com/earth-engine/tutorials/community/detecting-changes-in-sentinel-1-imagery-pt-2) of the Tutorial we will discuss statistical methods to detect changes in two Sentinel-1 images acquired at different times.